In [1]:
import numpy as np
import sounddevice as sd

In [96]:
def binarize_letter(x):
    i = ord(x)
    assert 0 < i < 128
    return list("{:07b}".format(i))

def frequentize_letter(x, n=7, mid=3, base=1):
    bits = binarize_letter(x)
    assert len(bits) == n
    return [base * (2 ** (i - mid)) for i, bit in enumerate(bits) if bit == "1"]

def frequentize(s):
    return [frequentize_letter(x) for x in list(s)]

frequentize("Hello world!")

[[0.125, 1],
 [0.125, 0.25, 2, 8],
 [0.125, 0.25, 1, 2],
 [0.125, 0.25, 1, 2],
 [0.125, 0.25, 1, 2, 4, 8],
 [0.25],
 [0.125, 0.25, 0.5, 2, 4, 8],
 [0.125, 0.25, 1, 2, 4, 8],
 [0.125, 0.25, 0.5, 4],
 [0.125, 0.25, 1, 2],
 [0.125, 0.25, 2],
 [0.25, 8]]

In [98]:
def play_tone(freqs_sets, duration, base_freq=440, sample_rate=44100, amplitude=0.5):
    # Generate time axis
    t = np.linspace(0, duration, int(sample_rate * duration), endpoint=False)
    
    # Generate waveform
    waveform = amplitude * np.concatenate([
        np.sum(
            [np.sin(2 * np.pi * freq * base_freq * t) for freq in freqs],
            axis=0
        )
        for freqs in freqs_sets
    ])
    
    # Play sound
    sd.play(waveform, sample_rate)
    
    # Wait for the sound to finish playing
    sd.wait()

# Example usage: Play a 440 Hz tone for 2 seconds
play_tone(
    frequentize('Hello world! This is a message from the deep'),
    0.1
)